## Replacing validataion_epoch_end with on_validation_epoch_end
* If you have a lightning v2.0.x installed, you need to use on_validation_epoch_end instread. 
* Here is a snipper code:
```
class LitModel(pl.LightningModule):
    def __init__(self):
        self.validation_step_outputs = []
        ...
    def validation_step(self, batch, batch_idx):
        self.validation_step_outputs.append(loss)
        ...
    def on_validation_epoch_end(self): 
        avg_loss = torch.stack(self.validation_step_outputs).mean()
        ...
```

In [4]:
import os
import torch
import pytorch_lightning as pl
#import lightning as pl
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
from torch.nn import functional as F

from torchmetrics import Accuracy

class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=10)
        self.validation_step_outputs = []
        

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        # logs metrics for each training_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        preds = torch.argmax(y_hat, dim=1)
        self.val_accuracy.update(preds, y)
        self.validation_step_outputs.append(loss)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True, on_step=True, on_epoch=True )
        self.log("val_acc", self.val_accuracy, prog_bar=True, on_step=True, on_epoch=True )
        return {'val_loss': loss}
   
    def on_validation_epoch_end(self): 
        avg_loss = torch.stack(self.validation_step_outputs).mean()
        #print("avg_loss: ", avg_loss)
        self.log("avg_val_loss", avg_loss, prog_bar=True)
        self.validation_step_outputs.clear()
        return {'avg_val_loss': avg_loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

BATCH_SIZE = 128 if torch.cuda.is_available() else 32
#train_loader = DataLoader(MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()))

train_loader = DataLoader(
    MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()), num_workers=4, 
    batch_size = BATCH_SIZE
)

val_loader = DataLoader(
    MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()), num_workers=4,
    batch_size = BATCH_SIZE
)

trainer = pl.Trainer(
    accelerator="auto",
    max_epochs = 5
)
model = LitModel()
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/scratch/qualis/miniconda3/envs/lightning/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytor

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
